In [1]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, WebDriverException
import json

In [2]:
 # Deals URL
URL = 'https://www.amazon.com/gp/goldbox?ref_=nav_cs_gb&discounts-widget=%2522%257B%255C%2522state%255C%2522%253A%257B%255C%2522refinementFilters%255C%2522%253A%257B%257D%257D%252C%255C%2522version%255C%2522%253A1%257D%2522'

In [3]:
 # Dictonary to save the scraped data
data2 =  { 'Product_ID' : [] ,'Product_Name' : [] ,'Category' : [] , 'Product_Link' : [] ,'Discounted_Price' : [] , 'Actual_Price' : [] , 'Rating' : [] , 'Rating_Count' : [] , 'Reviewers' : [] , 'Reviews' : []   }


In [4]:
productID_set = set()

In [5]:
 # scrolling in small increments
increment = 0.03  # Scroll by 3% of the remaining page height


In [ ]:
try:
     # opening the page 
    driver = webdriver.Chrome()
    driver.get( URL )
    driver.maximize_window()
    driver.execute_script("document.body.style.zoom='80%'")

    while True:
    
         # Create an explicit wait object
        wait = WebDriverWait(driver, 20)

         # Wait for an element to be present
        element = wait.until(EC.presence_of_all_elements_located(( By.CSS_SELECTOR, 'a[data-testid="product-card-link"]') ))

        html1 = driver.page_source

         # Creating a soup object 
        main_soup = BeautifulSoup( html1 , 'lxml' )

         # getting the div
        div_attr = main_soup.select('div:has(a[data-testid="product-card-link"])') 

        for elements in div_attr:

            Product_id = elements.get('data-asin')

            if Product_id in productID_set:
                continue

            else:
                 # Updating the Product Id set
                productID_set.add(Product_id)

                data2['Product_ID'].append(Product_id)

                 # Find the a element within this div
                a_element = elements.select_one('a[data-testid="product-card-link"]')
    
                # Get the href attribute from the a element
                if a_element:
                     # Getting the link and adding it to the data2
                    link = a_element.get('href')
                    data2['Product_Link'].append(link)

                    driver.get(link)
                    sleep(1)
                
                     # Getting the data using bs4
                    title = wait.until(EC.presence_of_element_located(( By.CSS_SELECTOR, 'span#productTitle') ))

                    if title:
                        # Getting the html 
                        html = driver.page_source

                         # Beautiful Soup object
                        soup_2 = BeautifulSoup( html , 'lxml' )
                    
                        # Getting the  actual prices
                        actual_price_attr = soup_2.select('span.aok-relative span.a-size-small.aok-offscreen')


                        if  actual_price_attr:
                   
                            actual = actual_price_attr[0].get_text().strip()

                            data2['Actual_Price'].append(actual)
                        

                        else:

                            data2['Actual_Price'].append('')

                         # Discounted price
                        discounted_price_attr = soup_2.select('span.a-price.aok-align-center span.a-offscreen')

                        if  discounted_price_attr:
                   
                            discount = discounted_price_attr[0].get_text().strip()

                            data2['Discounted_Price'].append(discount)

                        else:

                            data2['Discounted_Price'].append('')

                         # Getting the Product_name
                        name_attr = soup_2.select( 'span#productTitle') 

                        if name_attr:

                            name = name_attr[0].get_text().strip().split(',')
                            data2['Product_Name'].append(name[0])

                        else:

                            data2['Product_Name'].append('')

                         # Getting the Category
                        category_attr = soup_2.select( 'span.a-list-item a[href]' )

                        if category_attr:

                            category = category_attr[0].get_text().strip() + '$' + category_attr[1].get_text().strip()
                            data2['Category'].append(category)

                        else:

                            data2['Category'].append('')



                         # Getting rating and rating count 
                        rating_attr = soup_2.select('span[data-hook="rating-out-of-text"]')

                        if rating_attr:
                        
                            rating = rating_attr[0].get_text().strip()
                            data2['Rating'].append(rating)

                        else:

                            data2['Rating'].append('')

                        rating_count_attr = soup_2.select('span[data-hook="total-review-count"]')

                        if rating_count_attr:

                            count = rating_count_attr[0].get_text().strip()
                            data2['Rating_Count'].append(count)

                        else:

                            data2['Rating_Count'].append('')


                         # Getting the reviewer names 
                        reviewer_attr = soup_2.select( 'span.a-profile-name' )

                        reviewers = []

                        if reviewer_attr:

                            for i in reviewer_attr:
                                cleaned_text = i.get_text().strip()

                                if cleaned_text:
                                    reviewers.append(cleaned_text)

                            data2['Reviewers'].append('|'.join(reviewers[1:]))
            
                    
                        else:
                        
                            data2['Reviewers'].append('')

                         # Getting Review Title
                        reviews_attr = soup_2.select('a[data-hook="review-title"] span')

                        reviews = []
                        rating_keyword = "out of 5 stars"
    
                        if reviews_attr:
                            for i in reviews_attr:

                                cleaned_titles = i.get_text().strip()

                                if cleaned_titles and rating_keyword not in cleaned_titles:
                                    reviews.append(cleaned_titles)

                            data2['Reviews'].append('|'.join(reviews))

                        else:

                            data2['Reviews'].append('')
                            
                    driver.back()
                    sleep(1)

                else:
                    data2['Product_Link'].append('')
        
         # Scrolling the page
        scroll_position = driver.execute_script("return window.pageYOffset + window.innerHeight + window.innerHeight * {0}".format(increment))
    
         # Scroll by the incremented amount
        driver.execute_script(f"window.scrollTo(0, {scroll_position});")

         # Check and click the "Load More" button
        try:
            load_more_button = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="load-more-view-more-button"]')

            if load_more_button:
                
                load_more_button.click()
                print("Button clicked.")
                sleep(5.5)  # Wait for content to load

        except NoSuchElementException:
            
            print("Load More button not found. Continuing scroll.")

        

        if len(productID_set) >= 2000 : # break the loop when distinct data = 2000
            break
        
        
                
except WebDriverException as web_err:
    print(f'Selenium WebDriver error occurred: {web_err}')
except Exception as e:
    print(f'An error occurred: {e}')

finally:
    driver.quit()
        

In [8]:
# Save the set to a text file, treating None values explicitly
with open('productid.txt', 'w') as file:
    for id in productID_set:
        file.write(str(id) + '\n')  # Convert None to string and write to file


In [9]:
with open('Amazon_data1.json', mode='w', encoding='utf-8') as file:
    # Write the data2 dictionary to the file as JSON
    json.dump(data2, file, ensure_ascii=False, indent=4)